<a href="https://colab.research.google.com/github/graphlit/graphlit-samples/blob/main/python/Notebook%20Examples/Graphlit_2024_09_08_Describe_Related_Images_from_Web_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Description**

This example shows how to search the Web and ingest web pages, crawl image links, and describe the images. It also shows how to search the image descriptions based on the original Web search to find images related to your search text.

**Requirements**

Prior to running this notebook, you will need to [signup](https://docs.graphlit.dev/getting-started/signup) for Graphlit, and [create a project](https://docs.graphlit.dev/getting-started/create-project).

You will need the Graphlit organization ID, preview environment ID and JWT secret from your created project.

Assign these properties as Colab secrets: GRAPHLIT_ORGANIZATION_ID, GRAPHLIT_ENVIRONMENT_ID and GRAPHLIT_JWT_SECRET.


---

Install Graphlit Python client SDK

In [5]:
!pip install --upgrade graphlit-client

Initialize Graphlit

In [6]:
import os
from google.colab import userdata
from graphlit import Graphlit
from graphlit_api import input_types, enums, exceptions

os.environ['GRAPHLIT_ORGANIZATION_ID'] = userdata.get('GRAPHLIT_ORGANIZATION_ID')
os.environ['GRAPHLIT_ENVIRONMENT_ID'] = userdata.get('GRAPHLIT_ENVIRONMENT_ID')
os.environ['GRAPHLIT_JWT_SECRET'] = userdata.get('GRAPHLIT_JWT_SECRET')

graphlit = Graphlit()

Define Graphlit helper functions

In [7]:
from typing import List, Optional

# Create specification for OpenAI GPT-4o
async def create_openai_specification():
    if graphlit.client is None:
        return;

    input = input_types.SpecificationInput(
        name="OpenAI GPT-4o",
        type=enums.SpecificationTypes.EXTRACTION,
        serviceType=enums.ModelServiceTypes.OPEN_AI,
        openAI=input_types.OpenAIModelPropertiesInput(
            model=enums.OpenAIModels.GPT4O_128K,
        )
    )

    try:
        response = await graphlit.client.create_specification(input)

        return response.create_specification.id if response.create_specification is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

# Create specification for Anthropic Sonnet 3.5
async def create_anthropic_specification():
    if graphlit.client is None:
        return;

    input = input_types.SpecificationInput(
        name="Anthropic Claude Sonnet 3.5",
        type=enums.SpecificationTypes.EXTRACTION,
        serviceType=enums.ModelServiceTypes.ANTHROPIC,
        anthropic=input_types.AnthropicModelPropertiesInput(
            model=enums.AnthropicModels.CLAUDE_3_5_SONNET,
        )
    )

    try:
        response = await graphlit.client.create_specification(input)

        return response.create_specification.id if response.create_specification is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

# Create enrichment workflow to crawl only image links from web pages, and enrich them with Groq Llava 1.5
async def create_workflow(specification_id: str):
    if graphlit.client is None:
        return;

    input = input_types.WorkflowInput(
        name="Link Crawling",
        extraction=input_types.ExtractionWorkflowStageInput(
            jobs=[
                input_types.ExtractionWorkflowJobInput(
                    connector=input_types.EntityExtractionConnectorInput(
                        type=enums.EntityExtractionServiceTypes.MODEL_IMAGE,
                        modelImage=input_types.ModelImageExtractionPropertiesInput(
                            specification=input_types.EntityReferenceInput(id=specification_id)
                        )
                    )
                )
            ]
        ),
        enrichment=input_types.EnrichmentWorkflowStageInput(
            link=input_types.LinkStrategyInput(
                enableCrawling=True,
                allowContentDomain=True,
                allowedLinks=[enums.LinkTypes.FILE],
                allowedFiles=[enums.FileTypes.IMAGE],
            )
        )
    )

    try:
        response = await graphlit.client.create_workflow(input)

        return response.create_workflow.id if response.create_workflow is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

async def create_feed(search_text: str, read_limit: int, workflow_id: str):
    if graphlit.client is None:
        return;

    input = input_types.FeedInput(
        name="Web Search",
        type=enums.FeedTypes.SEARCH,
        search=input_types.SearchFeedPropertiesInput(
            text=search_text,
            readLimit=read_limit
        ),
        workflow=input_types.EntityReferenceInput(
            id=workflow_id
        )
    )

    try:
        response = await graphlit.client.create_feed(input)

        return response.create_feed.id if response.create_feed is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

async def is_feed_done(feed_id: str):
    if graphlit.client is None:
        return;

    response = await graphlit.client.is_feed_done(feed_id)

    return response.is_feed_done.result if response.is_feed_done is not None else None

# Locate images ingested by feed, optionally search by text within images
async def query_contents(feed_id: str, search: Optional[str] = None):
    if graphlit.client is None:
        return;

    try:
        response = await graphlit.client.query_contents(
            filter=input_types.ContentFilter(
                search=search,
                searchType=enums.SearchTypes.HYBRID,
                fileTypes=[enums.FileTypes.IMAGE],
                feeds=[
                    input_types.EntityReferenceFilter(
                        id=feed_id
                    )
                ]
            )
        )

        return response.contents.results if response.contents is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def delete_all_workflows():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_workflows(is_synchronous=True)


async def delete_all_feeds():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_feeds(is_synchronous=True)


Execute Graphlit example

In [8]:
from IPython.display import display, Markdown, Image
import time

# Remove any existing feeds and workflows; only needed for notebook example
await delete_all_workflows()
await delete_all_feeds()

print('Deleted all feeds.')

read_limit = 1 # how many search results to ingest from feed

search_text = "Seattle Kraken mascot Buoy"

# NOTE: uncomment just one of these to use different model
specification_id = await create_anthropic_specification()
#specification_id = await create_openai_specification()

if specification_id is not None:
    print(f'Created specification [{specification_id}].')

    workflow_id = await create_workflow(specification_id)

    if workflow_id is not None:
        print(f'Created workflow [{workflow_id}].')

        feed_id = await create_feed(search_text, read_limit, workflow_id)

        if feed_id is not None:
            print(f'Created feed [{feed_id}].')

            # Wait for feed to complete, since ingestion happens asychronously
            done = False
            time.sleep(5)
            while not done:
                done = await is_feed_done(feed_id)

                if not done:
                    time.sleep(2)

            print(f'Completed feed [{feed_id}].')

            # Show all the images crawled from the web page
            contents = await query_contents(feed_id)

            if contents is not None:
                print(f'Found {len(contents)} images in feed [{feed_id}].')
                print()

                for content in contents:
                    if content is not None and content.image is not None:
                        display(Image(url=content.image_uri, width=256))

            print()

            # Show just the images related to the search text, and the image text and descriptions
            contents = await query_contents(feed_id, search_text)

            if contents is not None:
                print(f'Found {len(contents)} images in feed [{feed_id}] related to [{search_text}].')
                print()

                for content in contents:
                    if content is not None:
                        display(Markdown(f'## {content.name} [{content.id}]:'))
                        display(Markdown(f'Original image size: {content.image.width}x{content.image.height}'))

                        if content.markdown is not None:
                            display(Markdown(f'Text in image: {content.markdown}'))

                        if content.image is not None:
                            display(Image(url=content.image_uri, width=512))

                            if content.image.description is not None:
                                display(Markdown(f'### Description:\n{content.image.description}'))

Deleted all feeds.
Created specification [0be82c03-8d4c-477a-918a-113aea379ca7].
Created workflow [b8a85ec1-3028-4f33-8b99-de6858e35a3a].
Created feed [64f797d4-51f2-4630-b1a5-01650b5afad4].
Completed feed [64f797d4-51f2-4630-b1a5-01650b5afad4].
Found 15 images in feed [64f797d4-51f2-4630-b1a5-01650b5afad4].




Found 4 images in feed [64f797d4-51f2-4630-b1a5-01650b5afad4] related to [Seattle Kraken mascot Buoy].



## ifttga66jgjxwb82fc58.png [26a8a73d-25a1-4ad9-9e8c-636247c16c24]:

Original image size: 720x405

Text in image: BUOY

0



...

### Description:
The image presents a distinctive logo design that combines elements of a popular social media platform's branding with custom text. The logo is horizontally oriented, featuring two main components: a stylized camera icon and text.

On the left side of the logo, we see a simplified representation of a camera. This icon is rendered in a gradient color scheme, transitioning from a vibrant pink or magenta at the top left to a warm orange or red at the bottom right. The camera design is minimalist, consisting of a rounded square shape with a smaller circle inside, likely representing the camera lens. This gradient effect gives the icon a modern, eye-catching appearance.

To the right of the camera icon, we find bold, sans-serif text that reads 'NHLBUOY'. The text is presented in a dark blue or navy color, creating a strong contrast against the colorful icon. The font choice appears to be clean and professional, enhancing readability while maintaining a contemporary feel.

The juxtaposition of the colorful, gradient-filled camera icon with the solid-colored text creates an interesting visual balance. This design choice effectively draws attention to both elements of the logo, ensuring that neither overpowers the other.

The use of the camera icon strongly suggests a connection to visual media, photography, or image-sharing platforms. This is further reinforced by the similarity of the icon to Instagram's well-known logo, which also features a simplified camera design with a gradient color scheme.

The text 'NHLBUOY' appears to be a unique identifier or brand name. The inclusion of 'NHL' could potentially reference the National Hockey League, while 'BUOY' might relate to maritime equipment or metaphorically to something that provides support or guidance. However, without additional context, the exact meaning or significance of this term remains open to interpretation.

Overall, this logo design effectively combines recognizable social media imagery with custom branding elements. It suggests a platform or service that may be related to visual content sharing, potentially with a focus on sports (particularly hockey) or marine-related themes. The professional execution of the design implies a legitimate and established brand identity.

## cbt37l3okfcw5ptksnes.png [5ad01b8e-7c57-488f-85a5-24da546d8afa]:

Original image size: 720x120

Text in image: SEABUOY



### Description:
The image presents a sleek and modern logo design that combines two distinct elements: a popular social media icon and a brand name. On the left side of the logo, we see the instantly recognizable TikTok logo - a musical note symbol rendered in a gradient of cyan and pink colors against a black background. This icon is widely associated with the short-form video sharing platform that has gained immense popularity in recent years.

To the right of the TikTok icon, separated by a vertical line, is the text 'SEABUOY' in bold, uppercase letters. The font used for 'SEABUOY' is a clean, sans-serif typeface that gives a professional and contemporary feel. The text is presented in a dark blue or navy color, which contrasts well with the vibrant TikTok icon and the black background.

The juxtaposition of the TikTok icon with the 'SEABUOY' text suggests a collaboration or association between the social media platform and what appears to be a brand or company named Seabuoy. This combination could indicate that Seabuoy is leveraging TikTok's platform for marketing, content creation, or some form of partnership.

The overall design of the logo is minimalist and impactful. The use of the well-known TikTok icon immediately draws attention and creates a sense of familiarity for viewers. The simple color palette of cyan, pink, navy, and black creates a visually striking effect while maintaining a professional appearance.

The vertical line separating the TikTok icon from the Seabuoy text serves as a clear divider, allowing each element to stand on its own while still forming a cohesive unit. This design choice helps to maintain the individual identity of Seabuoy while benefiting from the association with TikTok.

The choice to present this logo against a black background enhances its visibility and impact. The dark background allows the bright colors of the TikTok icon and the crisp text of 'SEABUOY' to stand out prominently, ensuring that the logo catches and holds the viewer's attention.

From a branding perspective, this logo effectively communicates a modern, tech-savvy image for Seabuoy. By aligning itself with TikTok, the brand positions itself as current and in touch with contemporary social media trends. This could be particularly appealing to younger demographics or businesses looking to expand their digital presence.

The simplicity of the design also lends itself well to versatility. This logo could easily be adapted for use across various digital and print media, maintaining its impact whether displayed on a small mobile screen or a large billboard.

While the exact nature of Seabuoy's business or its relationship with TikTok is not explicitly clear from the logo alone, the design successfully creates intrigue and prompts viewers to want to learn more about the brand and its connection to the popular social media platform.

In conclusion, this logo represents a clever fusion of established social media branding with a new or partnering entity. It effectively captures attention, conveys a modern aesthetic, and suggests a strategic alignment that could be beneficial for both TikTok and Seabuoy in their respective markets or collaborative ventures.